In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, SimpleRNN, Dropout , concatenate,  Input
from matplotlib import pyplot as plt

2022-11-06 13:29:57.794505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
DataSet = pd.read_csv('/Users/qasimk/Downloads/ML/data/data1.csv')
DataSet

,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,pm,stator_yoke,ambient,torque,profile_id
0,-0.450682,18.805172,19.086670,-0.350055,18.293219,0.002866,0.004419,0.000328,24.554214,18.316547,19.850691,1.871008e-01,17
1,-0.325737,18.818571,19.092390,-0.305803,18.294807,0.000257,0.000606,-0.000785,24.538078,18.314955,19.850672,2.454175e-01,17
2,-0.440864,18.828770,19.089380,-0.372503,18.294094,0.002355,0.001290,0.000386,24.544693,18.326307,19.850657,1.766153e-01,17
3,-0.327026,18.835567,19.083031,-0.316199,18.292542,0.006105,0.000026,0.002046,24.554018,18.330833,19.850647,2.383027e-01,17
4,-0.471150,18.857033,19.082525,-0.332272,18.291428,0.003133,-0.064317,0.037184,24.565397,18.326662,19.850639,2.081967e-01,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330811,-2.152128,30.721298,45.274497,0.791801,41.934347,0.000380,-2.000169,1.097528,62.147780,38.653720,23.989078,-8.116730e-60,71
1330812,-2.258684,30.721306,45.239017,0.778900,41.868923,0.002985,-2.000499,1.097569,62.142646,38.656328,23.970700,-5.815891e-60,71
1330813,-2.130312,30.721312,45.211576,0.804914,41.804819,0.002301,-1.999268,1.098765,62.138387,38.650923,23.977234,-4.167268e-60,71
1330814,-2.268498,30.721316,45.193508,0.763091,41.762220,0.005662,-2.000999,1.095696,62.133422,38.655686,24.001421,-2.985978e-60,71


In [4]:
Target = ['stator_winding', 'stator_tooth','stator_yoke' ,'torque' ]

X = DataSet[DataSet['profile_id'] != 20].drop(columns = [ 'stator_winding', 'stator_tooth','stator_yoke' ,'torque' ,'profile_id'])
y = DataSet[Target][DataSet['profile_id'] != 20]

X_test = DataSet[DataSet['profile_id'] == 20].drop(columns = ['stator_winding', 'stator_tooth','stator_yoke' ,'torque' ,'profile_id'])
y_test = DataSet[Target][DataSet['profile_id'] == 20]

scaler1 = StandardScaler()
scaler2 = StandardScaler()
X = scaler1.fit_transform(X)
y = scaler2.fit_transform(np.asarray(y).reshape(-1, 4))

X_test = scaler1.transform(X_test)
y_test = scaler2.transform(np.asarray(y_test).reshape(-1, 4))

X_train, X_rnn, y_train, y_rnn = train_test_split(X,y, test_size = 0.5,shuffle=False)

In [222]:
input_1 = Input(shape=(X.shape[1]))
hlay1_1 = Dense(16,activation = 'relu')(input_1)
hlay2_1 = Dense(16,activation = 'relu')(hlay1_1)
final_dnn1 = Dense(4,activation = 'linear')(hlay2_1)
dnn1 = Model(inputs=input_1, outputs=final_dnn1)
dnn1.compile(optimizer = "adam", loss = "mean_squared_error")

input_2 = Input(shape=(X.shape[1]))
hlay1_2 = Dense(32,activation = 'relu')(input_2)
hlay2_2 = Dense(16,activation = 'relu')(hlay1_2)
final_dnn2 = Dense(4,activation = 'linear')(hlay2_2)
dnn2 = Model(inputs=input_2, outputs=final_dnn2)
dnn2.compile(optimizer = "adam", loss = "mean_absolute_percentage_error")

input_3 = Input(shape=(X.shape[1]))
hlay1_3 = Dense(16,activation = 'sigmoid')(input_3)
hlay2_3 = Dense(16,activation = 'sigmoid')(hlay1_3)
hlay3_3 = Dense(8,activation = 'relu')(hlay2_3)
final_dnn3 = Dense(4,activation = 'linear')(hlay3_3)
dnn3 = Model(inputs=input_3, outputs=final_dnn3)
dnn3.compile(optimizer = "adam", loss = "mean_squared_error")

input_4 = Input(shape=(X.shape[1]))
hlay1_4 = Dense(32,activation = 'relu')(input_4)
hlay2_4 = Dense(32,activation = 'sigmoid')(hlay1_4)
final_dnn4 = Dense(4,activation = 'linear')(hlay2_4)
dnn4 = Model(inputs=input_4, outputs=final_dnn4)
dnn4.compile(optimizer = "adam", loss = "mean_squared_error")

dnn1.save("dnn1")
dnn2.save("dnn2")
dnn3.save("dnn3")
dnn4.save("dnn4")


INFO:tensorflow:Assets written to: dnn1/assets
INFO:tensorflow:Assets written to: dnn2/assets
INFO:tensorflow:Assets written to: dnn3/assets
INFO:tensorflow:Assets written to: dnn4/assets


In [ ]:
dnn1 = keras.models.load_model("dnn1")
dnn2 = keras.models.load_model("dnn2")
dnn3 = keras.models.load_model("dnn3")
dnn4 = keras.models.load_model("dnn4")

dnn1.trainable = False; dnn2.trainable = False; dnn3.trainable = False; dnn4.trainable = False;

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2)

enn_input = Input(shape=(X_train.shape[1]))
const_1 = dnn1(enn_input)
const_2 = dnn2(enn_input)
const_3 = dnn3(enn_input)
const_4 = dnn4(enn_input)
enn_1 = Dense(8,activation = 'relu')(enn_input)
merge = concatenate(inputs = [const_1,const_2,const_3,const_4,enn_1])
enn_2 = Dense(32,activation = 'relu')(merge)
enn_3 = Dense(32,activation = 'relu')(enn_2)
final = Dense(4,activation = 'linear')(enn_3)
enn = Model(inputs=enn_input, outputs=final)
enn.compile(optimizer = "adam", loss = "mean_squared_error")

enn.save("enn")

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_rnn,y_rnn, test_size = 0.2)

enn.trainable = False;

input_rnn = Input(shape=(X_train.shape[1], 1))
input_enn = Input(shape=(X_train.shape[1]))
rnn1 = SimpleRNN(16, return_sequences=True)(input_rnn)
drop1 = Dropout(0.2)(rnn1)
rnn2 = SimpleRNN(16, return_sequences=False)(drop1)
drop2 = Dropout(0.2)(rnn2)
enn_lay = enn(input_enn)
merge = concatenate(inputs = [enn_lay,drop2])
combine1 = Dense(16,activation = 'relu')(merge)
combine2 = Dense(16,activation = 'relu')(combine1)
final = Dense(4)(combine2)

ernn = Model(inputs=[input_rnn,input_enn], outputs=final)
ernn.compile(optimizer = "adam", loss = "mean_squared_error")
ernn.save("ernn")

INFO:tensorflow:Assets written to: ernn/assets
